In [1]:

import os
from werkzeug.wsgi import SharedDataMiddleware
from flask import Flask, render_template, request, jsonify
from flask_login import LoginManager, login_user, logout_user, login_required, UserMixin
import requests
import bcrypt

app = Flask(__name__)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
app.config['SECRET_KEY'] = "lkkajdghdadkglajkgah"

@login_manager.user_loader
def load_user(user_id):
    return User(user_id)

class User(UserMixin):
  def __init__(self,id):
    self.id = id

@app.route("/")
@login_required
def home():
    user = {"name":"Bernard"}
    return render_template('home.html',message="")

@app.route("/list_users")
def table():
    headers = {
        'Authorization': 'Bearer keyRfjHV4Hm1sCpg5',
    }

    r = requests.get('https://api.airtable.com/v0/app5BsEkn1GleGb5f/flaskdemo?sortField=_createdTime&sortDirection=desc', headers=headers)
    dict = r.json()
    dataset = []
    for i in dict['records']:
         dict = i['fields']
         dataset.append(dict)
    return render_template('table.html', entries=dataset)

@app.route("/login")
def login():
    message = 'Please login in first.'
    return render_template('login.html', message=message)

@app.route("/user")
@login_required
def user():
    message = 'Please enter user information.'
    return render_template('userform.html', message=message)

@app.route("/adduser",methods=['POST'])
@login_required
def adduser():
    fname = request.form['fname']
    lname = request.form['lname']
    student_id = request.form['student_id']
    date_of_birth = request.form['date_of_birth']
    pwd = request.form['pwd']
    pwd = pwd.encode('UTF-8')
    # Hash a password for the first time, with a randomly-generated salt
    hashed = bcrypt.hashpw(pwd, bcrypt.gensalt())
    pwd = hashed.decode('UTF-8')
    mydict =  {
        "fname": fname,
        "lname": lname,
        "student_id": student_id,
        "date_of_birth": date_of_birth,
        'pwd': pwd
    }
    data = {"fields": mydict}
    headers = {'Authorization': 'Bearer keyRfjHV4Hm1sCpg5', 'Content-Type': 'application/json; charset=utf-8'}
    r = requests.post('https://api.airtable.com/v0/app5BsEkn1GleGb5f/flaskdemo',json=data,headers=headers)
    message = 'Please enter user information.'
    return render_template('home.html',message=message)

@app.route("/updateuser",methods=['POST','PUT'])
@login_required
def updateuser():
    record_id = request.form['record_id']
    headers = {
        'Authorization': 'Bearer keyRfjHV4Hm1sCpg5',
    }
    r = requests.get('https://api.airtable.com/v0/app5BsEkn1GleGb5f/flaskdemo/' + record_id, headers=headers)
    dict = r.json()
    dict_list = dict['fields']
    for i in dict_list:
        if (i == 'pwd'):
            pwd = dict_list[i]

    fname = request.form['fname']
    lname = request.form['lname']
    student_id = request.form['student_id']
    date_of_birth = request.form['date_of_birth']

    fields =  {
    "fname": fname,
    "lname": lname,
    "student_id": student_id,
    "date_of_birth": date_of_birth,
    "pwd": pwd
    }

    data = {
      "records": [
          {
          "id": record_id,
          "fields": fields
          }
      ]
    }

    headers = {'Authorization': 'Bearer keyRfjHV4Hm1sCpg5', 'Content-Type': 'application/json; charset=utf-8'}
    r = requests.put('https://api.airtable.com/v0/app5BsEkn1GleGb5f/flaskdemo',json=data,headers=headers)
    message = 'Please enter user information.'
    return render_template('home.html',message=message)

@app.route("/deleteuser",methods=['POST','DELETE'])
@login_required
def deleteuser():
    url = "https://api.airtable.com/v0/app5BsEkn1GleGb5f/flaskdemo/"
    record_id = request.form['record_id']
    headers = {'Authorization': 'Bearer keyRfjHV4Hm1sCpg5', 'Content-Type': 'application/x-www-form-urlencoded'}
    r = requests.delete(url + record_id, headers=headers)
    message = 'Please enter user information.'
    return render_template('home.html',message=message)

@app.route("/process",methods=['POST'])
def process():
    student_id = request.form['student_id']
    password = request.form['password']
    password = password.encode('UTF-8')
    hashed = ''
    user = ''
    headers = {
    'Authorization': 'Bearer keyRfjHV4Hm1sCpg5',
    }

    # filter = 'IF(AND({UserName}="'+username+'",{Pwd}="'+password+'"), 1, 0)'
    filter = 'IF(({student_id}="'+student_id+'"), 1, 0)'

    params = (
        ('view', 'Grid view'),
        ('filterByFormula', filter),
    )

    dataset = []
    r = requests.session()
    r = requests.get('https://api.airtable.com/v0/app5BsEkn1GleGb5f/flaskdemo?sortField=_createdTime&sortDirection=desc', headers=headers, params=params)

    dict = r.json()

    for i in dict['records']:
         dict = i['fields']
         dataset.append(dict)
    for item in dataset:
        user = item.get('student_id')
        if user != None:
            hashed = item.get('pwd')
            fname = item.get('fname')
            hashed = hashed.encode('UTF-8')
        else:
            message = 'wrong password!'
            return render_template('login.html',message=message)

    if  student_id == user and bcrypt.checkpw(password, hashed):
        login_user(User(1))
        user = {"name":"Bernard"}
        message = "Dear " + fname + ", welcome to Bernard's pages. Your login has been granted."
        return render_template('home.html', user=user, title="home page",message=message)
    message = 'wrong password!'
    return render_template('login.html',message=message)

@app.errorhandler(500)
def internal_error(error):
    message = 'wrong password!'
    return render_template('login.html',message=message),500

@app.route('/logout/')
@login_required
def logout():
    logout_user()
    message = 'Thanks for logging out.'
    return render_template('login.html',message=message)
#@app.route('/')
#def hello_world():
#     return 'Hello from Flask!'

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9013, app)

ModuleNotFoundError: No module named 'flask_login'